<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Procesamiento de lenguaje natural
## Desafio N°2

Valentín Pertierra

#### Consigna:

**1**. Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.  

**2**. Probar términos de interés y explicar similitudes en el espacio de embeddings.  

**3**. Intentar plantear y probar tests de analogías.  

**4**. Graficar los embeddings resultantes.  

**5**. Sacar conclusiones.

In [ ]:
!pip install ebooklib
!pip install beautifulsoup4 lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ebooklib: filename=EbookLib-0.18-py3-none-any.whl size=38778 sha256=48471329421e9347174ed299f6fbdf3804725547b4909cc454d13e2dd7d9a82a
  Stored in directory: /root/.cache/pip/wheels/0f/38/cc/a3728bb72a315d9d8766fb71d362136372066fc25ad838f8fa
Successfully built ebooklib


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

from tensorflow.keras.preprocessing.text import text_to_word_sequence

from IPython.display import display, Markdown
import pandas as pd

In [ ]:
from ebooklib import epub, ITEM_DOCUMENT
from bs4 import BeautifulSoup

### Datos
Utilizare como dataset el libro de "El señor de los anillos".

In [ ]:
# Monto el drive en el notebook para leer el libro (esto si estoy desde el colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/My Drive/LSE/NLP/Desafios/Desafio 2/El Señor de los Anillos.epub'
book = epub.read_epub(path)

/usr/local/lib/python3.10/dist-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')


In [ ]:
# Obtengo el texto del epub
book_text = ''

for item in book.get_items():
  if item.get_type() == ITEM_DOCUMENT:
    content = item.get_content()
    soup = BeautifulSoup(content, 'html.parser')
    book_text += soup.get_text()

In [ ]:
# Muestro parte del texto
book_text[0:2000]

'\n\n\n\n\n\n\n\n\n\n\n\xa0\nEl Señor de los Anillos es una novela, obra cumbre del escritor John Ronald Reuel Tolkien, ambientada en un mundo fantástico llamado la Tierra Media hacia el final de su Tercera Edad. El título hace referencia a Sauron, el Señor Oscuro de Mordor, principal villano de la historia, creador del Anillo Único que utilizó para controlar el poder de los demás Anillos. El Señor de los Anillos constituye la continuación de un libro anterior de Tolkien, El Hobbit, que cuenta la historia de cómo el Anillo del Poder pasa a las manos de Bilbo Bolsón, el tío de Frodo Bolsón. En un nivel más profundo, constituye la continuación de otro libro llamado El Silmarillion, que habla de la creación de la Tierra Media y de todas las criaturas que en ella habitan, así como del primer Señor Oscuro, maestro de Sauron, y de las luchas por los Silmarils.\nEl Señor de los Anillos narra las aventuras de un grupo de seres: (elfos, hobbits, enanos, humanos), que forman la Comunidad del Ani

### 1) Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

#### Preprocesamiento del texto

In [ ]:
# Armo el dataset utilizando los . como separador de oraciones
sentence_tokens = []

for sentence in book_text.split('.'):
  # Agrego filto de caracteres para elimnar en particular — « » y otros caracteres frecuentes
  sentence_tokens.append(text_to_word_sequence(sentence, filters='¡!"#$%&()*+,-—«»./:;<=>¿?@[\\]^_`{|}~\t\n\r'))

sentence_tokens = [sentence for sentence in sentence_tokens if sentence]

In [ ]:
for sentence in sentence_tokens[:10]:
  print(sentence)

['\xa0', 'el', 'señor', 'de', 'los', 'anillos', 'es', 'una', 'novela', 'obra', 'cumbre', 'del', 'escritor', 'john', 'ronald', 'reuel', 'tolkien', 'ambientada', 'en', 'un', 'mundo', 'fantástico', 'llamado', 'la', 'tierra', 'media', 'hacia', 'el', 'final', 'de', 'su', 'tercera', 'edad']
['el', 'título', 'hace', 'referencia', 'a', 'sauron', 'el', 'señor', 'oscuro', 'de', 'mordor', 'principal', 'villano', 'de', 'la', 'historia', 'creador', 'del', 'anillo', 'único', 'que', 'utilizó', 'para', 'controlar', 'el', 'poder', 'de', 'los', 'demás', 'anillos']
['el', 'señor', 'de', 'los', 'anillos', 'constituye', 'la', 'continuación', 'de', 'un', 'libro', 'anterior', 'de', 'tolkien', 'el', 'hobbit', 'que', 'cuenta', 'la', 'historia', 'de', 'cómo', 'el', 'anillo', 'del', 'poder', 'pasa', 'a', 'las', 'manos', 'de', 'bilbo', 'bolsón', 'el', 'tío', 'de', 'frodo', 'bolsón']
['en', 'un', 'nivel', 'más', 'profundo', 'constituye', 'la', 'continuación', 'de', 'otro', 'libro', 'llamado', 'el', 'silmarillion',

#### Creación de los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# Creo distintos modelos para luego comparar resultados

# En este caso utilizaremos la estructura modelo Skipgram con windows 3
w2v_model_sk_3 = Word2Vec(min_count=5,       # frecuencia mínima de palabra para incluirla en el vocabulario
                          window=3,          # cant de palabras antes y desp de la predicha
                          vector_size=300,   # dimensionalidad de los vectores
                          negative=20,       # cantidad de negative samples... 0 es no se usa
                          workers=1,         # si tienen más cores pueden cambiar este valor
                          sg=1)              # modelo 0:CBOW  1:skipgram

# Modelo Skipgram con windows 5
w2v_model_sk_5 = Word2Vec(min_count=5,       # frecuencia mínima de palabra para incluirla en el vocabulario
                          window=5,          # cant de palabras antes y desp de la predicha
                          vector_size=300,   # dimensionalidad de los vectores
                          negative=20,       # cantidad de negative samples... 0 es no se usa
                          workers=1,         # si tienen más cores pueden cambiar este valor
                          sg=1)              # modelo 0:CBOW  1:skipgram

# Modelo CBOW con windows 3
w2v_model_cb_3 = Word2Vec(min_count=5,       # frecuencia mínima de palabra para incluirla en el vocabulario
                          window=3,          # cant de palabras antes y desp de la predicha
                          vector_size=300,   # dimensionalidad de los vectores
                          negative=20,       # cantidad de negative samples... 0 es no se usa
                          workers=1,         # si tienen más cores pueden cambiar este valor
                          sg=0)              # modelo 0:CBOW  1:skipgram

# Modelo CBOW con windows 3
w2v_model_cb_5 = Word2Vec(min_count=5,       # frecuencia mínima de palabra para incluirla en el vocabulario
                          window=5,          # cant de palabras antes y desp de la predicha
                          vector_size=300,   # dimensionalidad de los vectores
                          negative=20,       # cantidad de negative samples... 0 es no se usa
                          workers=1,         # si tienen más cores pueden cambiar este valor
                          sg=0)              # modelo 0:CBOW  1:skipgram


In [ ]:
# Obtener el vocabulario con los tokens
w2v_model_sk_3.build_vocab(sentence_tokens)
w2v_model_sk_5.build_vocab(sentence_tokens)
w2v_model_cb_3.build_vocab(sentence_tokens)
w2v_model_cb_5.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_sk_3.corpus_count)

Cantidad de docs en el corpus: 31384


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model_sk_3.wv.index_to_key))

Cantidad de words distintas en el corpus: 7233


#### Entrenamiento de embeddings

In [ ]:
# Entreno los modelos
# Utilizamos nuestro callback
w2v_model_sk_3.train(sentence_tokens,
                    total_examples=w2v_model_sk_3.corpus_count,
                    epochs=30,
                    compute_loss = True,
                    callbacks=[callback()]
                    )

Loss after epoch 0: 4432383.0
Loss after epoch 1: 3326357.0
Loss after epoch 2: 3172896.0
Loss after epoch 3: 3096239.0
Loss after epoch 4: 3041281.0
Loss after epoch 5: 2921778.0
Loss after epoch 6: 2886938.0
Loss after epoch 7: 2848802.0
Loss after epoch 8: 2808656.0
Loss after epoch 9: 2781032.0
Loss after epoch 10: 2763482.0
Loss after epoch 11: 2780380.0
Loss after epoch 12: 2731052.0
Loss after epoch 13: 2690192.0
Loss after epoch 14: 2656304.0
Loss after epoch 15: 2613012.0
Loss after epoch 16: 2588544.0
Loss after epoch 17: 2553752.0
Loss after epoch 18: 2523980.0
Loss after epoch 19: 2504080.0
Loss after epoch 20: 2483476.0
Loss after epoch 21: 2461416.0
Loss after epoch 22: 2450592.0
Loss after epoch 23: 2074976.0
Loss after epoch 24: 444848.0
Loss after epoch 25: 428704.0
Loss after epoch 26: 422800.0
Loss after epoch 27: 406072.0
Loss after epoch 28: 393432.0
Loss after epoch 29: 386280.0


(10504017, 15831930)

In [ ]:
w2v_model_sk_5.train(sentence_tokens,
                    total_examples=w2v_model_sk_5.corpus_count,
                    epochs=30,
                    compute_loss = True,
                    callbacks=[callback()]
                    )

Loss after epoch 0: 5320718.0
Loss after epoch 1: 4514332.0
Loss after epoch 2: 4323531.0
Loss after epoch 3: 4219173.0
Loss after epoch 4: 4097852.0
Loss after epoch 5: 4051720.0
Loss after epoch 6: 4014714.0
Loss after epoch 7: 4009548.0
Loss after epoch 8: 4103832.0
Loss after epoch 9: 4057948.0
Loss after epoch 10: 4002004.0
Loss after epoch 11: 3956580.0
Loss after epoch 12: 3922800.0
Loss after epoch 13: 3866680.0
Loss after epoch 14: 3839040.0
Loss after epoch 15: 3812848.0
Loss after epoch 16: 1522984.0
Loss after epoch 17: 709888.0
Loss after epoch 18: 696536.0
Loss after epoch 19: 676856.0
Loss after epoch 20: 660800.0
Loss after epoch 21: 644584.0
Loss after epoch 22: 623424.0
Loss after epoch 23: 603232.0
Loss after epoch 24: 585600.0
Loss after epoch 25: 563064.0
Loss after epoch 26: 545480.0
Loss after epoch 27: 525896.0
Loss after epoch 28: 507192.0
Loss after epoch 29: 489008.0


(10501784, 15831930)

In [ ]:
w2v_model_cb_3.train(sentence_tokens,
                    total_examples=w2v_model_cb_3.corpus_count,
                    epochs=30,
                    compute_loss = True,
                    callbacks=[callback()]
                    )

Loss after epoch 0: 1644594.5
Loss after epoch 1: 1146208.0
Loss after epoch 2: 1012083.25
Loss after epoch 3: 930087.25
Loss after epoch 4: 877926.5
Loss after epoch 5: 857582.0
Loss after epoch 6: 842075.5
Loss after epoch 7: 825115.5
Loss after epoch 8: 783718.5
Loss after epoch 9: 757101.0
Loss after epoch 10: 746331.0
Loss after epoch 11: 733722.0
Loss after epoch 12: 723196.0
Loss after epoch 13: 712270.0
Loss after epoch 14: 703946.0
Loss after epoch 15: 694727.0
Loss after epoch 16: 686266.0
Loss after epoch 17: 678832.0
Loss after epoch 18: 670224.0
Loss after epoch 19: 661945.0
Loss after epoch 20: 606413.0
Loss after epoch 21: 590776.0
Loss after epoch 22: 584114.0
Loss after epoch 23: 579280.0
Loss after epoch 24: 573884.0
Loss after epoch 25: 568892.0
Loss after epoch 26: 562884.0
Loss after epoch 27: 560228.0
Loss after epoch 28: 555626.0
Loss after epoch 29: 551132.0


(10504017, 15831930)

In [ ]:
w2v_model_cb_5.train(sentence_tokens,
                    total_examples=w2v_model_cb_5.corpus_count,
                    epochs=30,
                    compute_loss = True,
                    callbacks=[callback()]
                    )

Loss after epoch 0: 1598853.5
Loss after epoch 1: 1162301.5
Loss after epoch 2: 1032735.0
Loss after epoch 3: 949826.0
Loss after epoch 4: 897291.0
Loss after epoch 5: 879410.5
Loss after epoch 6: 863320.5
Loss after epoch 7: 849538.0
Loss after epoch 8: 805416.0
Loss after epoch 9: 784861.0
Loss after epoch 10: 772259.0
Loss after epoch 11: 762955.0
Loss after epoch 12: 753312.0
Loss after epoch 13: 743385.0
Loss after epoch 14: 734180.0
Loss after epoch 15: 726190.0
Loss after epoch 16: 717045.0
Loss after epoch 17: 708932.0
Loss after epoch 18: 701884.0
Loss after epoch 19: 667073.0
Loss after epoch 20: 632506.0
Loss after epoch 21: 626888.0
Loss after epoch 22: 619706.0
Loss after epoch 23: 613132.0
Loss after epoch 24: 607294.0
Loss after epoch 25: 600798.0
Loss after epoch 26: 597386.0
Loss after epoch 27: 591284.0
Loss after epoch 28: 586724.0
Loss after epoch 29: 584092.0


(10504965, 15831930)

### 2) Probar términos de interés y explicar similitudes en el espacio de embeddings.

In [ ]:
words = ['anillo', 'poder', 'sauron', 'espada', 'sméagol', 'frodo']
models = [w2v_model_sk_3, w2v_model_sk_5, w2v_model_cb_3, w2v_model_cb_5]

# Comparo las palabras que mas se relacionan utilizando los distintos modelos
for word in words:

  display(Markdown(f'Palabras similares a **{word}:**\n'))
  rows = []

  for model in models:

    result = model.wv.most_similar(positive=[word], topn=10)

    row = [f'{sim_word} ({score:.2f})' for sim_word, score in result]
    rows.append(row)

  df = pd.DataFrame(rows, index=['Skipgram w3','Skipgram w5', 'CBOW w3', 'CBOW w5'])

  display(df)
  print('\n')


Palabras similares a **anillo:**


,0,1,2,3,4,5,6,7,8,9
Skipgram w3,dedo (0.46),único (0.40),bolsillo (0.38),refiere (0.38),dejaría (0.37),tenerlo (0.37),miembro (0.37),busco (0.36),fabricado (0.36),misericordia (0.35)
Skipgram w5,dedo (0.47),dejaría (0.41),bolsillo (0.39),portador (0.39),huye (0.38),celebrimbor (0.38),siervos (0.38),pusiera (0.38),miembro (0.38),único (0.38)
CBOW w3,bolsillo (0.50),enemigo (0.47),destino (0.45),dedo (0.45),título (0.43),paquete (0.42),cargo (0.41),concilio (0.41),objeto (0.40),tesoro (0.40)
CBOW w5,enemigo (0.49),destino (0.44),bolsillo (0.44),tesoro (0.43),objeto (0.42),dedo (0.41),sauron (0.40),paquete (0.40),poder (0.39),papel (0.39)


Palabras similares a **poder:**


,0,1,2,3,4,5,6,7,8,9
Skipgram w3,provecho (0.41),renunciar (0.41),dominio (0.40),poderío (0.39),sauron (0.39),luchar (0.39),acompañan (0.38),devorado (0.37),habláis (0.37),tenerlo (0.37)
Skipgram w5,acompañan (0.39),devorado (0.38),provecho (0.37),rivalizar (0.37),llegarían (0.35),curar (0.35),infortunio (0.35),maldad (0.34),malicia (0.34),sauron (0.34)
CBOW w3,dominio (0.52),valor (0.51),poderío (0.48),territorio (0.48),enemigo (0.46),sauron (0.45),destino (0.44),ataque (0.44),plan (0.44),pensamiento (0.44)
CBOW w5,dominio (0.53),sauron (0.53),valor (0.50),enemigo (0.46),poderío (0.46),malicia (0.46),territorio (0.44),destino (0.43),voluntad (0.43),temor (0.42)


Palabras similares a **sauron:**


...

,0,1,2,3,4,5,6,7,8,9
Skipgram w3,vaina (0.47),desprendió (0.47),esgrimía (0.47),desenvainó (0.46),empuñaba (0.46),glamdring (0.46),andúril (0.45),empuñadura (0.45),mellada (0.45),reluciente (0.43)
Skipgram w5,vaina (0.51),rota (0.49),andúril (0.49),desenvainó (0.48),glamdring (0.48),empuñadura (0.48),mellada (0.47),desprendió (0.46),esgrimía (0.46),empuñaba (0.45)
CBOW w3,empuñadura (0.60),vara (0.59),mano (0.58),vaina (0.58),hoja (0.55),pala (0.54),lanza (0.54),silla (0.53),garra (0.52),manta (0.52)
CBOW w5,empuñadura (0.64),vara (0.61),vaina (0.61),mano (0.59),lanza (0.57),andúril (0.57),hoja (0.56),flecha (0.55),soltando (0.54),dardo (0.54)


Palabras similares a **sméagol:**


,0,1,2,3,4,5,6,7,8,9
Skipgram w3,tesssoro (0.44),tesoro (0.43),sss (0.43),importa (0.43),creéis (0.41),seguid (0.40),cuidadoso (0.40),déagol (0.39),advierto (0.39),descubriste (0.39)
Skipgram w5,sss (0.41),déagol (0.40),siéntate (0.38),tesoro (0.38),tesssoro (0.37),enigmas (0.37),importa (0.36),cuidadoso (0.36),tonterías (0.36),gimió (0.36)
CBOW w3,amo (0.49),precio (0.48),otho (0.47),gollum (0.46),quién (0.45),tesoro (0.45),déagol (0.44),gildor (0.42),alguien (0.42),realmente (0.42)
CBOW w5,amo (0.52),tesoro (0.50),quién (0.49),déagol (0.45),otho (0.45),gollum (0.45),eh (0.42),miserable (0.42),gildor (0.41),bueno (0.41)


Palabras similares a **frodo:**


,0,1,2,3,4,5,6,7,8,9
Skipgram w3,ironía (0.44),burlón (0.44),soñoliento (0.42),molesto (0.41),horrorizado (0.40),cabeceaba (0.39),duerma (0.39),aferrándose (0.39),sss (0.39),sam (0.39)
Skipgram w5,sam (0.50),ironía (0.42),burlón (0.41),aferrándose (0.40),sss (0.39),horrorizado (0.39),impaciente (0.39),rosita (0.39),respirando (0.39),severidad (0.39)
CBOW w3,mantecona (0.53),faramir (0.53),pippin (0.53),bilbo (0.53),merry (0.51),denethor (0.49),bergil (0.48),haldir (0.47),gollum (0.47),trancos (0.47)
CBOW w5,bilbo (0.54),pippin (0.53),mantecona (0.51),bergil (0.51),gollum (0.50),trancos (0.50),denethor (0.47),faramir (0.47),merry (0.46),sam (0.45)


En general se observan resultados que tienen conceptos similares a las palabras analizadas. Si bien hay diferencias entre los distintos modelos, mantienen una lógica similar, en varios casos se comparten las palabras mas similares.  

### 3) Intentar plantear y probar tests de analogías.

Se realizaron varias pruebas de analogías entre palabras, las siguientes son las que presentaron una mejor relación entre los conceptos

In [ ]:
w2v_model_cb_5.wv.most_similar(w2v_model_cb_5.wv.get_vector("hombre") - w2v_model_cb_5.wv.get_vector("alto"))

[('hombre', 0.6365919709205627),
 ('hobbit', 0.40678662061691284),
 ('mendigo', 0.40618041157722473),
 ('mediano', 0.36876389384269714),
 ('mandar', 0.3471773862838745),
 ('mensaje', 0.3430081009864807),
 ('daros', 0.3426133990287781),
 ('bob', 0.3375934362411499),
 ('arrastrar', 0.3371388018131256),
 ('amo', 0.3298795521259308)]

En este caso al restarle la palabra "alto" a "hombre" se obtiene como primer resultado más similar la palabra "hobbit" (excluyendo a "hombre"). Esto tienen mucho sentido ya que los hobbits tienen las artísticas fisionómicas de los hombres pero son de menor estatura.

In [ ]:
w2v_model_cb_5.wv.most_similar(w2v_model_cb_5.wv.get_vector("aragorn") - w2v_model_cb_5.wv.get_vector("hombre") +  w2v_model_cb_5.wv.get_vector("elfo"))

[('aragorn', 0.7041378617286682),
 ('gimli', 0.5476470589637756),
 ('celeborn', 0.4690888524055481),
 ('legolas', 0.41778695583343506),
 ('tristemente', 0.39788302779197693),
 ('haldir', 0.3974582254886627),
 ('elfo', 0.39423057436943054),
 ('ingold', 0.38815227150917053),
 ('imrahil', 0.37543800473213196),
 ('beregond', 0.36550936102867126)]

En este caso al restarle lo “hombre” a “aragorn” y sumarle lo “elfo” se obtiene como mas similar “gimli”, que es un enano, aunque “celeborn”, “legolas” y “haldir” son elfos. Lo que se puede resaltar es que al restarle una raza a un nombre propio y sumarle otra raza, en general se obtuvo otro nombre propio de otra raza.

In [ ]:
w2v_model_cb_5.wv.most_similar(w2v_model_cb_5.wv.get_vector("galadriel") - w2v_model_cb_5.wv.get_vector("elfo") + w2v_model_cb_5.wv.get_vector("mujer"))

[('galadriel', 0.7605605125427246),
 ('eowyn', 0.46636247634887695),
 ('salud', 0.43865901231765747),
 ('dama', 0.4322454631328583),
 ('mujer', 0.4285377860069275),
 ('hermosa', 0.4225112199783325),
 ('estrella', 0.394334077835083),
 ('bella', 0.39208492636680603),
 ('frasco', 0.38979968428611755),
 ('repentina', 0.38421329855918884)]

Este caso es un buen ejemplo de una analogía, porque al restarle lo “elfo” a “galadriel” y sumarle lo “mujer”, nos da como resultado más similar “eowyn”, que es una noble guerrera. No solo se obtiene un buen resultado al restarle una raza y sumarle otra, sino que también son del mismo género (esto también paso en el cado de restar elfo y sumar hombre).   

In [ ]:
w2v_model_cb_5.wv.most_similar(w2v_model_cb_5.wv.get_vector("rohan") - w2v_model_cb_5.wv.get_vector("hombre") + w2v_model_cb_5.wv.get_vector("sauron"))

[('rohan', 0.6126441359519958),
 ('sauron', 0.6026455760002136),
 ('mordor', 0.4838304817676544),
 ('umbar', 0.46807825565338135),
 ('angmar', 0.4506990313529968),
 ('gondor', 0.44827958941459656),
 ('númenor', 0.4385078251361847),
 ('harad', 0.43723565340042114),
 ('huestes', 0.41582998633384705),
 ('arnor', 0.41449326276779175)]

En este caso intente a una locación, sacarle los habitantes y agregarle otros, para obtener como resultado otra locación. Rohan es un reino y al restarle “hombre” y sumarle a “sauron” se obtiene como resultado “mordor”, que son las tierras de Sauron. También se obtiene como resultado otras locaciones como umbar, angmar, gondor, númenor, harad y arnor.

### 4) Graficar los embeddings resultantes.

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

In [ ]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model_cb_5)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab")

In [ ]:
# Graficar los embedddings en 3D
vecs, labels = reduce_dimensions(w2v_model_cb_5, 3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab")

En los graficos se pueden apreciar diferentes cúmulos de palabras agrupadas. En particular se puede mencionar el caso de las palabras “hobbits”, “orcos”, “hombres” y “elfos”, donde se muestra la cercanía entre estas palabras al tratarse de diferentes razas.   

Las palabras “norte”, “sur” y “oeste” se encuentran muy juntos, casi en la misma posición, aunque “este” esta cerca, no esta tan junto como los otros puntos cardinales, esto puede ser porque la palabra “este” tienen muchas mas connotaciones que un punto cardinal.  

Se puede apreciar un gran cumulo de nombres propios como “gandalf”, “boromir”, “merry”, “sam”, “saruman”, entre otros.


In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model_cb_5.wv.vectors)
labels = list(w2v_model_cb_5.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### 6) Conclusiones

En este desafío se utilizaron dos modelos word2vect para generar embedding de los términos presentes en un corpus, en particular en el libro del Señor de los Anillos.

Al analizar la similitud entre los embeddings obtenidos en ambos modelos, Skipgram y CBOW, se observa que logran aprender los conceptos de los términos, mostrando cercania entre términos que conceptualmente están cerca.  

Al realizar un análisis de analogías, es decir, obtener un termino a partir de la operación algebraica entre vectores, se encontraron casos donde el resultado es muy satisfactorio, mostrando buenos ejemplo entre la distancia entre razas y nombres propios de personajes. Esto se puede corroborar al realizar gráficos de los embedding, donde conceptos similares tienen una menor distancia.

